# Projet 3 de BioInformatique
Rémy Detobel - 000408013                
[rdetobel@ulb.ac.be](mailto:rdetobel@ulb.ac.be)        

------------------------

## Table des matières

- [Introduction](#Introduction)
- [Méthode](#Méthode)
    - [Données](#Données)
    - [Implémentation](#Implémentation)
        - [Import](#Import)
        - [Profil](#Profil)
        - [Align](#Align)
- [Résultats](#Résultats)

## Introduction
Ce projet vise à construire un profil pour la famille de séquences WW.  Ce profil sera ensuite utilisé pour aligner des séquences (données par le professeur).  Ces résultats seront ensuite comparés aux données disponibles sur les bases de données en ligne.              
`Python` par l'intermédiaire de `Jupyter` sera utilisé pour développer un outil permettant de calculer ce profil et de réaliser l'alignement grâce à celui-ci.

## Méthode
#### Données
Avant tout il nous faut récupérer la famille de séquences `WW`.  Pour cela on va utiliser la base de données [SMART](http://smart.embl.de) qui va nous permettre de récupérer le fichier [to-be-aligned.fasta](Data/to-be-aligned.fasta).  Ce fichier est ensuite aligné en utilisant les outils [MUSCLE](http://www.ebi.ac.uk/Tools/msa/muscle/) et [CLUSTAL Omega](http://www.ebi.ac.uk/Tools/msa/clustalo/) (il est également possible de le faire avec [TCoffee](http://www.ebi.ac.uk/Tools/msa/tcoffee/)).  Les résultats de ces alignements sont enregistrés dans des fichiers, respectivement: [msaresults-MUSCLE.fasta](Data/msaresults-MUSCLE.fasta) et [msaresults-CLUSTAL_Omega.fasta](Data/msaresults-CLUSTAL_Omega.fasta) (mais également [msaresults-TCoffee.fasta](Data/msaresults-TCoffee.fasta)).                 
On récupère également la proportion totale de chaque acide aminé sur toutes les bases de données que contient SwissProt.  Pour cela on se rend sur le site [expasy](http://web.expasy.org) et plus précisément sur [cette page](http://web.expasy.org/docs/relnotes/relstat.html).  Les informations nous intéressant se trouvent au point 6.1. Ces données seront directement écrites dans le code vu l'impossibilité d'exporter correctement cette page (à noter également que les données sont en pourcentages).

In [1]:
SWISS_PROT = {'A':0.0826, 'R':0.0553, 'N':0.0406, 'D':0.0546, 'C':0.0137,
              'Q':0.0393, 'E':0.0674, 'G':0.0708, 'H':0.0227, 'I':0.0593, 
              'L':0.0965, 'K':0.0582, 'M':0.0241, 'F':0.0386, 'P':0.0472, 
              'S':0.0660, 'T':0.0534, 'W':0.0109, 'Y':0.0292, 'V':0.0687}

#### Implémentation

###### Import
Comme dit dans la partie [Données](#Données), il faut récupérer plusieurs informations dont certaines se trouvant dans des fichiers.             
La fonction `importSeq` permet d'extraire une liste de séquences d'un fichier.

In [2]:
def importSeq(nom_fichier):

    # Ouverture du fichier
    fichier = open(nom_fichier, 'r')

    listeSeq = []
    currentSeq = ""

    for ligne in fichier:
        ligne = ligne.strip()

        if(len(ligne) > 0):
            if(ligne[0] == '>'):
                if(currentSeq != ""):
                    listeSeq.append(currentSeq)

                currentSeq = ""
            else:
                currentSeq += ligne

    if(currentSeq != ""):
        listeSeq.append(currentSeq)

    return listeSeq

###### Profil
L'objet Profil représente un groupe de séquences et enregistre les propriétés générales d'une collection de séquences.  Dans le cas présent, il permet également de faire une abstraction du problème et plus précisément des calculs permettant de produire le PSSM.              
À sa construction, l'objet profil parcourt toutes les séquences qu'il va représenter et calcule la fréquence de chaque acide aminé en fonction de la colonne dans laquelle il est présent:
$$\text{Frequence}(i, aa) = \frac{\text{Nbr aa à la colonne i}}{\text{nbr séquence}}$$
Où `aa` représente un acide aminé et `i` représente l'indice d'une colonne.         

On va ensuite pondérer ces fréquences grâce aux données récupérées sur SwissProt ainsi que via des _pseudocounts_.      
Les _pseudoscounts_ sont des constantes que l'on rajoute et qui permettent de donner une valeur même lorsqu'aucun acide aminé n'est présent pour une certaine colonne.  Concrètement, cela revient à faire ce calcul:
$$q(u, a) = \frac{\alpha \text{ fréquence}(u, a) + \beta p_{a}}{\alpha + \beta}$$
Où $\beta$ et $\alpha$ sont des _pseudocounts_ et valent:
- $\beta = \sqrt{N_{seq}}$
- $\alpha = \text{nombre de séquences ayant un AA à cette position}$
- $p_{a}$ est la valeur présente dans SwissProt pour un acide aminé `a`
           
Les _pseudocounts_ représente la distribution antérieure du système avant de faire le calcul avec les données.  Ici nous nous basons sur le nombre de séquence, mais il est également possible de calculer les _pseudocounts_ en utilisant une matrice de substitution.  Cette technique n'a pas été mise en place ici pour des raisons de simplicité mais également car les résultats finaux n'étaient pas très différents.


Enfin, il ne reste plus que le calcul du PSSM qui se fait via la formule suivante:
$$m_{u, a} = \log \frac{q_{u, a}}{p_a}$$
Où
- $m_{u,a}$ est la valeur à mettre dans notre PSSM pour la colonne `u` et l'acide aminé `a`
- $q_{u,a}$ est la probabilité calculée juste avant
- $p_a$ est la valeur présente dans SwissProt (comme dans la précédente formule)

Toutes ces opérations sont faites automatiquement à la création de l'objet Profil.  Lors de cette création, il est également possible de choisir la valeur qu'aura un trou/gap.            
Pour récupérer le PSSM créé par ce Profil il suffit d'appeler la méthode `getPSSM` qui renverra une liste de dictionnaire calculée lors de la création de l'objet.

In [3]:
from math import sqrt
from math import log10
# Just for design
from IPython.display import display, HTML


class Profil:
    
    # Initialisation en fonction d'une liste de séquences
    def __init__(self, listeSeq, penalite = -1):
        self.listeSeq = listeSeq
        # Caclul du pseudoCount Beta
        self.pseudoCountBeta = sqrt(len(listeSeq))
        
        self.penalite = penalite
        
        print("Nombre de séquence: "+str(len(listeSeq)))
        self.freqPosition()      # Calcul la fréquence de chaque aa à une colonne donnée
        self.calculProbabilite() # Calcul de la probabilité d'avoir un aa à une colonne
        self.calculPSSM()        # Calcul du PSSM

    
    # Nombre d'AA à une certaine colonne
    def freqPosition(self):
        self.totalElemCol = [0]

        # Nombre de lettres par colonne
        self.listeFreq = [{}]
        
        # Pour toutes les séquences
        for seq in self.listeSeq:
            col = 0 # Pour chaque colonne
            for lettre in seq:
                # Initialise les listes si c'est une nouvelle colonne
                if(len(self.listeFreq) <= col):
                    self.listeFreq.append({})
                    self.totalElemCol.append(0)
                
                # Si la lettre est déjà présente
                if(lettre in self.listeFreq[col]):
                    self.listeFreq[col][lettre] += 1
                    self.totalElemCol[col] += 1

                elif(lettre != '-'): # Sinon
                    self.listeFreq[col][lettre] = 1
                    self.totalElemCol[col] += 1
                    
                col += 1

        # Calcul de la fréquence
        total = len(self.listeSeq) # Nombre total de séquences
        for colonne in self.listeFreq:
            for lettre in colonne:
                colonne[lettre] = colonne[lettre]/total

    
    # Création d'une liste contenant toutes les valeurs des probabilités
    def calculProbabilite(self):
        self.listeProba = [{}]
        col = 0
        
        for colonne in self.listeFreq:
            if(len(self.listeProba) <= col):
                self.listeProba.append({})

            for lettre in SWISS_PROT:
                self.listeProba[col][lettre] = self.makeCalcul(lettre, colonne, col)

            col += 1

    
    # Calcul en tant que tel des probabilités
    def makeCalcul(self, lettre, colonne, nbrColonne):
        # \frac{\alpha F(i,j) + \beta p(i)}{\alpha + \beta}

        valeurColonne = 0
        if(lettre in colonne):
            valeurColonne = colonne[lettre]

        pseudoCountAlpha = self.totalElemCol[nbrColonne]

        # Numérateur
        num = pseudoCountAlpha * valeurColonne + self.pseudoCountBeta * SWISS_PROT[lettre]
        # Dénominateur
        den = pseudoCountAlpha + self.pseudoCountBeta

        return (num / den)


    def calculPSSM(self):
        # m(i,j) = log \frac{q(i,j)}{p(i)}
        self.pssm = [{}]

        col = 0
        for colonne in self.listeProba:
            if(len(self.pssm) <= col):
                self.pssm.append({})

            for lettre in colonne:
                self.pssm[col][lettre] = log10(colonne[lettre]/SWISS_PROT[lettre])
            self.pssm[col]['-'] = self.penalite

            col += 1
            

    def getPSSM(self):
        return self.pssm;
    
    
    # Permet de récupérer la séquence formée avec le AA ayant le plus de chance d'être trouvée
    def getSeqType(self):
        res = ""
        for colonne in self.pssm:
            res += max(colonne, key=colonne.get)
            
        return res
    
    def printSeqType(self):
        seqType = self.getSeqType()
        printRes = [""]
        printCol = [""]
        printValeur = [""]

        col = 1
        i = 0
        for seq in seqType:
            if(col%20 == 0):
                printRes.append("")
                printCol.append("")
                printValeur.append("")
                i += 1

            printRes[i] += seq + "  "
            printCol[i] += (str(col) + " " if col >= 10 else str(col) + "  ")
            if(self.pssm[col-1][seq] > 0.9):
                printValeur[i] += "+  "
            else:
                printValeur[i] += "   "
            col+=1

        for j in range(len(printRes)):
            print(printRes[j])
            print(printCol[j])
            print(printValeur[j])
            print("")
    
        

Essayons maintenant de comparer un PSSM avec un WebLogo

In [4]:
print("Création d'un profil pour MUSCLE avec une pénalité de 0.2")
profilMuscle = Profil(importSeq("Data/msaresults-MUSCLE.fasta"), -0.2)
print("") # Ajout d'un espace

profilMuscle.printSeqType()

Création d'un profil pour MUSCLE avec une pénalité de 0.2
Nombre de séquence: 247

P  L  P  P  G  W  E  M  R  W  D  R  P  D  C  G  R  V  Y  
1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 
      +     +  +  +           +              +        +  

Y  Y  N  F  T  L  V  A  Q  A  H  S  S  Q  V  L  C  G  E  N  
20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
+     +                                                     

D  T  R  E  S  V  P  S  H  N  T  R  T  T  Q  W  E  H  P  R  
40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
                        +     +        +     +        +     

W  
60 
   



Voici le WebLOGO de Muscle:
![WebLogoMuscle](./Images/MUSCLE_weblogo.png)

On remarque bien que la séquence type concorde parfaitement avec le web logo.  Cela se voit très fortement en position 2, 3, 4 et 5 par exemple où l'on trouve respectivement L, P, G et W.
      
Les `+` représentent les lettres qui ont un PSSM supérieur à 0.9.  On remarque bien que le WebLogo est fidèle à ces indications.      


Cette comparaison a été faite avec le fichier MUSCLE, mais il est également possible de le faire avec Clustal Omega.

In [5]:
print("Création d'un profil pour Clustal Omega avec une pénalité de 0.2")
profilClustal = Profil(importSeq("Data/msaresults-CLUSTAL_Omega.fasta"), -0.2)
print("") # Ajout d'un espace

profilClustal.printSeqType()

Création d'un profil pour Clustal Omega avec une pénalité de 0.2
Nombre de séquence: 247

P  L  P  P  G  W  E  M  R  W  D  P  S  N  G  R  V  Y  Y  
1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 
      +     +  +  +           +           +        +  +  

Y  N  H  N  T  Q  K  V  C  G  E  N  W  T  R  L  G  S  L  Q  
20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 
   +  +     +                                               

E  S  V  P  S  Y  N  H  I  N  R  T  T  Q  W  E  H  P  R  W  
40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 
                                    +     +        +        



![Clustal_Omega](./Images/Omega_weblogo.png)
Là encore, on remarque que les lettre les plus grandes sont bien celles ayant la plus grande probabilité dans le PSSM.               
          
Cependant, il est intéressant de comparer les deux PSSM entre eux.  On peut voir beaucoup de similarités dans les deux WebLogo notamment au début et à la fin.


Il est également intéressant de le comparer avec le `HHM` de la famille WW.
![HHM_logo](./Images/HHM_Logo.png)

Ce HHM contient bien tous les éléments étants communs avec Omega et Clustal.  On remarque plusieurs grandes lettres qui sont également présentes dans les deux logos précédents comme par exemple le `L P G W D G Y Y N T W P` qui se trouvent respectivement en position: 1, 2, 4, 5, 10, 14, 17, 18, 20, 23, 28 et 31.
        
Au niveau des différences on peut remarquer que le Muscle néglige le H en position 23, tandis que Omega l'exagère (en position 22) par rapport au HHM.

###### Align
La class `Align` permet de fractionner le code afin de rendre l'outil plus lisibile.  `Align` permet donc d'aligner une chaine d'acides aminés en fonction du PSSM.  Ces deux informations sont donc passées en paramètre lors de la création de l'objet.  Celui-ci s'occupe ensuite de faire tous les calculs permettants d'aligner cette séquence.   
      
La création de l'objet commence donc par l'initialisation d'une matrice dont le nombre de lignes correspond à la taille de la séquence à alignée plus un (la première ligne étant une ligne utilisée pour remplir la matrice).  Le nombre de colonnes, lui, sera dépendant du nombre de colonnes dans le PSSM plus un (pour les mêmes raisons que les lignes).  La première  ligne et la première colonne sont donc initialisés avec des zéros.  En effet, c'est la méthode à suivre lorsque l'on veut faire un alignement local comme ici.       
Une fois la matrice créée on va la remplir.  Pour ce faire, la technique utilisée sera comparable au premier projet réalisé pour ce cours, à savoir calculer la valeur d'une case `i,j` en fonction des cases au-dessus, à gauche et en diagonale.  Mathématiquement cela se présente de la manière suivante:         
Il faut faire le maximum entre:
1. $S(i-1, j-1) + PSSM(j, seq[i])$               
  _Calculé depuis la diagonale + pssm_
2. $S(i-1, j) + PSSM(j, '-')$               
  _Calculé grâce à la case au-dessus + pssm_
3. $S(i, j-1) + PSSM(j-1, '-')$                
  _Calculé grâce à la case à gauche + pssm_

Le résultat ayant la plus grande valeur est donc utilisé pour la case $(i,j)$.           
Pendant que l'on remplit la matrice, une seconde matrice est également complétée.  Il s'agit d'une matrice stockant les déplacements possibles à partir d'une case.  En effet, lorsque l'on utilise une des 3 formules ci-dessus on effectue un déplacement:
1. En diagonale (_noté D_)
2. Verticalement (_noté V_)
3. Horizontalement (_noté H_)

Cette matrice de déplacement nous permettra d'éviter de faire des calculs plus tard.       
Lors de la construction on enregistre également les coordonnées de la valeur la plus grande.  En effet, comme il s'agit d'un alignement local, le backtracking permettant de trouver l'alignement se fera à partir de la valeur la plus grande de la matrice.  L'enregistrer ici permet donc d'éviter de reparcourir toute la matrice par la suite.            

Une fois tous les calculs faits, il ne reste plus qu'à parcourir la matrice de déplacement en partant des cordonnées de la plus grande valeur de la matrice (attention, il est possible que la valeur la plus grande se retrouve plusieurs fois dans la matrice et dans ce cas précis, plusieurs alignements sont alors possibles).   

In [6]:
class Align:

    # Création de l'objet
    def __init__(self, chaineA, pssm, debug = False):

        # Initialisation des variables
        self.chaineA = chaineA
        self.pssm = pssm
        self.debug = debug
        
        # Initialisation des matrices de calcul
        self.S = []
        # Creation d'une matrice ou la liste en ij permettra d'indiquer dans quel
        # direction on peut se déplacer (Horizontal, Vertical ou Diagonale)
        self.deplacement = []
        
        # Valeur maximum du tableau
        self.maxValue = 0
        # Coordonnées ou l'on peut trouver cette valeur
        self.maxI = [] 
        self.maxJ = []

        self.initMatrice()       # Initialisation de la matrice
        self.remplisageMatrice() # Remplissage de la matrice
        
        self.res = []   # Résultat final
        self.minIJ = [] # Coordonnées les plus petites atteintes
        
        # Parcours de la matrice pour trouver le plus court chemin
        for allMax in range(len(self.maxI)): # Parcourt toutes les valeurs maximum
            self.res += (self.findMinPath(self.maxI[allMax], self.maxJ[allMax], [], []))


    # Permet d'initialiser la matrice
    def initMatrice(self):
        
        for i in range(len(self.chaineA)+1):
            # Initialisation des lignes
            self.S.append([])
            self.deplacement.append([])

            for j in range(len(self.pssm)+1):
                if(i == 0): # Si l'on est sur la première ligne
                    # Initialisation de la matrice
                    self.S[i].append(0)
                    self.deplacement[i].append([])

                elif(j == 0): # Si l'on est sur la première colonne
                    # Initialisation de la matrice
                    self.S[i].append(0)
                    self.deplacement[i].append([])
                    
                else: # Sinon on remplit juste la liste des deplacements
                    self.deplacement[i].append([])


    # Permet de remplir la matrice
    def remplisageMatrice(self):
        # On parcourt toute la matrice
        for i in range(1, len(self.chaineA)+1):        # ligne
            for j in range(1, len(self.pssm)+1): # colonne

                valeur1 = self.S[i-1][j-1] + self.pssm[j-1][self.chaineA[i-1]]
                valeur2 = self.S[i-1][j] + self.pssm[j-1]['-']
                valeur3 = self.S[i][j-1] + self.pssm[j-2]['-']

                maximum = max(valeur1, valeur2, valeur3, 0)

                self.S[i].append(maximum)

                if(self.S[i][j] == valeur1):
                    self.deplacement[i-1][j-1].append("D") # Diagonale

                if(self.S[i][j] == valeur2):
                    self.deplacement[i-1][j].append("V") # Vertical

                if(self.S[i][j] == valeur3):
                    self.deplacement[i][j-1].append("H") # Horizontal


                # Enregistrement de la valeur maximum
                # Si la valeur est la même que le max actuel
                if(self.maxValue == maximum):
                    self.maxI.append(i)
                    self.maxJ.append(j)

                # Si le max est plus grand que la valeur actuelle
                elif(maximum > self.maxValue):
                    self.maxI = [i]
                    self.maxJ = [j]
                    self.maxValue = maximum
                

        
        if(self.debug): # Si on veut débug, on affiche les matrices intermédiaires
            self.printMatrice(self.S)
            print()
            self.printMatrice(self.deplacement)
    
    


    # Permet d'afficher une matrice
    def printMatrice(self, matrice):
        for i in range(len(matrice)):
            # Ligne avec les indices
            if(i == 0):
                print("     ", end="")
                for j in range(len(matrice[i])):
                    print('{:4}|'.format(j), end="")
                print()
            
            # Affichage des lignes
            for j in range(len(matrice[i])):
                if(j == 0):
                    # Affiche l'indice
                    print('{:4}|'.format(i), end="")

                if(isinstance(matrice[i][j], list)):
                    print('{:4}|'.format("".join(matrice[i][j])), end="")
                else:
                    print('{:4}|'.format(str(round(matrice[i][j], 2))), end="")
            print() # Retour à la ligne

    
    # Permet de trouver le(s) bon(s) chemin(s) dans la matrice
    def findMinPath(self, i, j, current=[], sol=[]):
        if(i == 0 and j == 0): # Si on a fini le parcours
            sol.append(list(reversed(current)))
            self.minIJ.append((i, j))

        else:
            continuer = False # Est-il possible de continuer
            
            # Si on peut encore monter et que ce déplacement est possible
            if(i > 0 and "V" in self.deplacement[i-1][j]):
                continuer = True
                current.append("V") # Construction d'une solution
                sol = self.findMinPath(i-1, j, current[:], sol)
                current = current[:-1] # Déconstruction
            
            # Test à gauche cette fois
            if(j > 0 and "H" in self.deplacement[i][j-1]):
                continuer = True
                current.append("H")
                sol = self.findMinPath(i, j-1, current[:], sol)
                current = current[:-1]

            # Et enfin en diagonale
            if(i > 0 and j > 0 and "D" in self.deplacement[i-1][j-1]):
                continuer = True
                current.append("D")
                sol = self.findMinPath(i-1, j-1, current[:], sol)
                current = current[:-1]
            
            # Si il n'est plus possible de continuer et qu'on est en local
            if(not continuer):
                # On enregistre donc la solution
                sol.append(list(reversed(current)))
                self.minIJ.append((i, j))

        return sol


    # Permet d'afficher toutes les solutions
    def printResult(self):
        # Parcours de toutes les solutions
        for indexDeplacement in range(len(self.res)):

            position = [""]
            resSeq = ["       "]
            modification = ["       "]
            i = 0
            total = 0
            

            # colonne et ligne de début à 0 sauf si on est en local
            col, ligne = self.minIJ[indexDeplacement]

            position[0] += str(col)+" |"
            
            # On parcourt le résultat
            for dep in self.res[indexDeplacement]:
                
                if(dep == "D"):
                    resSeq[i] += self.chaineA[col-1]
                    modification[i] += " "
                    # On déplace le curseur
                    ligne += 1
                    col += 1

                elif(dep == "V"):
                    resSeq[i] += self.chaineA[col-1]
                    modification[i] += "+"
                    col += 1

                elif(dep == "H"):
                    resSeq[i] += " "
                    modification[i] += "-"
                    ligne += 1
                    
                
                if(total > 0 and ((col-1) % 10 == 0) and dep != 'H'):
                    if(col-1 < 10):
                        position[i] += "  " + str(col-1)
                    elif(col < 100):
                        position[i] += " " + str(col-1)
                    else:
                        position[i] += str(col-1)
                        
                    if((col-1)%20 == 0):
                        position.append("")
                        resSeq.append("  ")
                        modification.append("  ")
                        i += 1
                    else:
                        position[i] += " "
                        resSeq[i] += "   "
                        modification[i] += "   "
                        
                else:
                    position[i] += " "


                total += 1

            position[i] += "   | "+str(col)

            # Affichage des solutions
            for j in range(len(position)):
                print(position[j])
                print(resSeq[j])
                print(modification[j])
                print("")
                
            if(indexDeplacement < (len(self.res)-1)):
                display(HTML('<hr />'))
            

    def getResult(self):
        return self.res
    

## Résultats

Maintenant que toutes les structures ont été définies, nous pouvons charger le fichier `test.fasta` qui contient deux séquences:

In [7]:
testSequence = importSeq("Data/test.fasta")

Ces deux séquences seront alignées une première fois avec le PSSM de Muscle et ensuite avec le PSSM contruit grâce à Clustal.  Ces deux PSSM ont été construits précédemment (dans la section [Profil](#Profil)).

In [8]:
alignMuscleUn = Align(testSequence[0], profilMuscle.getPSSM())
alignMuscleUn.printResult()

141 |         150             160
       VPLPPGWEMA   K TP SGQ RYFL
                     -  - + -    

                                  170      | 173
                           NHIDQTTTWQ   DP
  -------------------------               



In [9]:
alignClustalUn = Align(testSequence[0], profilClustal.getPSSM())
alignClustalUn.printResult()

141 |         150            160
       VPLPPGWEMA   KTP SGQ RYFL
                       - + -    

                                  170      | 173
  NH           I              DQTTTWQ   DP
    ----------- --------------            



141 |         150            160
       VPLPPGWEMA   KTP SGQ RYFL
                       - + -    

                                  170      | 173
  NH            I             DQTTTWQ   DP
    ------------ -------------            



141 |         150            160
       VPLPPGWEMA   KTP SGQ RYFL
                       - + -    

                                  170      | 173
  NH             I            DQTTTWQ   DP
    ------------- ------------            



141 |         150            160
       VPLPPGWEMA   KTP SGQ RYFL
                       - + -    

                                  170      | 173
  NH              I           DQTTTWQ   DP
    -------------- -----------            



141 |         150            160
       VPLPPGWEMA   KTP SGQ RYFL
                       - + -    

                                  170      | 173
  NH               I          DQTTTWQ   DP
    --------------- ----------            



141 |         150            160
       VPLPPGWEMA   KTP SGQ RYFL
                       - + -    

                                  170      | 173
  NH                      I   DQTTTWQ   DP
    ---------------------- ---            



On voit que l'alignement ne change pas beaucoup entre MUSCLE et Clustal Omega.  Ils commencent tous les deux au 141ème acide aminé et finissent au 173ème.           
On peut également comparer ces résultats à ceux disponibles sur Uniprot.  Plus précisément [ici](http://www.uniprot.org/blast/?about=D6C652[141-174]&key=Domain) où l'on peut voir que la séquence `D6C652` à des points communs avec WW entre sa 141ème et ça 174ème protéine:
```
>sp|D6C652|141-174
VPLPPGWEMAKTPSGQRYFLNHIDQTTTWQDPRK
```
Notre résultat n'est donc pas très éloigné.

**Notons cependant** que nous avons pris une pénalité très petite.  Cela est dû aux variables de SUISSPROT ainsi que des autres valeurs utilisées pour calculer les PSSM qui utilisent des logarithmes.         

Sur uniprot on peut remarquer qu'une autre partie de la séquence à des points communs avec la famille WW.  Pour trouver ce second alignement avec notre PSSM il faudrait appliquer un alignement sous optimal (en mettant toutes les valeurs trouvées pour faire l'alignement à zéro et en refaisant le backtracking).  Cette option n'a pas été développée car la structure actuelle du code ne permet pas son implémentation de manière simple (en effet, la matrice de déplacement est calculée à la construction de la matrice.  Faire l'alignement sous optimal implique de recalculer également cette matrice, ce qui complexifie fortement l'algorithme si l'on ne veut pas repartir d'une base vide).

In [10]:
alignMuscleDeux = Align(testSequence[1], profilMuscle.getPSSM())
alignMuscleDeux.printResult()

460 |          470                                      480
       GPLPPGWEERT   H  T DGRVFFI                         N
                 +    -- -       ------------------------- 

          490      | 493
  H NIKKTQWED   PR
   -+             



460 |          470                                      480
       GPLPPGWEERT   H  T DGRVFFI                         N
                 +    -- -       ------------------------- 

          490      | 493
  HN IKKTQWED   PR
   +-             



In [11]:
alignClustalUn = Align(testSequence[1], profilClustal.getPSSM())
alignClustalUn.printResult()

460 |          470            480
       GPLPPGWEERT   H  TDGRVFFIN
                 +    --         

                                  490      | 493
  H           N              IKKTQWED   PR
   ----------- --------------             



460 |          470            480
       GPLPPGWEERT   H  TDGRVFFIN
                 +    --         

                                  490      | 493
  H            N             IKKTQWED   PR
   ------------ -------------             



460 |          470            480
       GPLPPGWEERT   H  TDGRVFFIN
                 +    --         

                                  490      | 493
  H             N            IKKTQWED   PR
   ------------- ------------             



460 |          470            480
       GPLPPGWEERT   H  TDGRVFFIN
                 +    --         

                                  490      | 493
  H              N           IKKTQWED   PR
   -------------- -----------             



460 |          470            480
       GPLPPGWEERT   H  TDGRVFFIN
                 +    --         

                                  490      | 493
  H               N          IKKTQWED   PR
   --------------- ----------             



460 |          470            480
       GPLPPGWEERT   H  TDGRVFFIN
                 +    --         

                                  490      | 493
  H                      N   IKKTQWED   PR
   ---------------------- ---             



Les tests précédents ont été réalisés en utilisant la seconde séquence présente dans le fichier de test.  Il s'agit de la séquence `P46935`.         
Comme pour la séquence précédente, on remarque que l'alignement est presque le même pour Clustal que pour Muscle.  Il se rapproche également très fort de ce qui se trouve sur [uniprot](http://www.uniprot.org/blast/?about=P46935[460-493]&key=Domain), à savoir:
```
>sp|P46935|460-493
GPLPPGWEERTHTDGRVFFINHNIKKTQWEDPRL
```
Comme pour l'alignement précédent, uniprot nous informe que d'autres parties de la séquence ont des points communs avec la famille WW.  Pour les trouver il faudrait donc (comme pour la séquence précédente) faire un alignement sous-optimal.        
            
Le code ci-dessus rajoute des `-` et des `+`.  Il s'agit en fait de la traduction de déplacements verticaux ou horizontaux.  Les `-` faisant référence à un élément ayant disparu entre la séquence qui est alignée et le modèle.  Le `+` indique les acides aminés ayant été rajoutés dans la séquence par rapport au modèle.